# Software Requirements Classification
## Naive Bayes

In [1]:
import pandas as pd

df = pd.read_csv('software_requirements_extended.csv')
df['Type'] = df['Type'].apply(lambda x: 1 if x == 'FR' or x == 'F' else 0)

In [2]:
import re
import nltk  #natural language processing

nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nickbres/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
pattern  = re.compile(r"[^a-zA-Z0-9]")
def req_to_words(req):
    ''' Convert tweet text into a sequence of words '''
    # convert to lower case
    text = req.lower()
    # remove non letters
    text = re.sub(pattern, " ", text)
    # tokenize
    words = text.split()
    # remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    # apply stemming
    words = [PorterStemmer().stem(w) for w in words]
    # return list
    return words

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

### Model

In [8]:
y = df['Type'] # Target
X = [' '.join(req_to_words(req)) for req in df['Requirement']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [9]:
vectorizer2 = TfidfVectorizer(analyzer=req_to_words)
X_train = vectorizer2.fit_transform(X_train)
X_test = vectorizer2.transform(X_test)

In [11]:
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [13]:
y_pred = model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test,y_pred) * 100:.2f}%')
print(classification_report(y_test, y_pred))

Accuracy: 85.42%
              precision    recall  f1-score   support

           0       0.87      0.80      0.83       176
           1       0.85      0.90      0.87       215

    accuracy                           0.85       391
   macro avg       0.86      0.85      0.85       391
weighted avg       0.85      0.85      0.85       391
